In [1]:
# Instalação necessária
!pip install tensorflow_decision_forests

import pandas as pd
import tensorflow_decision_forests as tfdf
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.4 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
  Attempting uninstal

In [2]:
# Carregar os dados tratados
df = pd.read_csv("Data/Stroke.csv")

# Separar X e y
X = df.drop('stroke', axis=1)
y = df['stroke']

In [3]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [4]:
# Reunir novamente para uso no TFDF
train_df = X_train.copy()
train_df['stroke'] = y_train

test_df = X_test.copy()
test_df['stroke'] = y_test

# Converter para dataset do TFDF
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="stroke")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, label="stroke")

In [5]:
# Modelo Random Forest
model = tfdf.keras.RandomForestModel(num_trees=100, random_seed=42)
model.fit(train_ds)

Use /tmp/tmp76izmurw as temporary training directory
Reading training dataset...
Training dataset read in 0:00:08.149887. Found 7572 examples.
Training model...
Model trained in 0:00:04.535574
Compiling model...
Model compiled.


In [6]:
# Avaliação
evaluation = model.evaluate(test_ds, return_dict=True)
print(f"Avaliação no conjunto de teste: {evaluation}")

2/2 [==============================] - 1s 37ms/step - loss: 0.0000e+00
Avaliação no conjunto de teste: {'loss': 0.0}


In [7]:
# Predição
y_pred_proba = model.predict(test_ds)
y_pred = (y_pred_proba > 0.5).astype(int).reshape(-1)

2/2 [==============================] - 0s 29ms/step


In [8]:
# Relatórios
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))


Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      0.92      0.96       951
           1       0.93      1.00      0.96       943

    accuracy                           0.96      1894
   macro avg       0.96      0.96      0.96      1894
weighted avg       0.96      0.96      0.96      1894

Matriz de Confusão:
 [[875  76]
 [  0 943]]
